In [ ]:
import random
import os
from tqdm import tqdm
import torch
from PIL import Image
import numpy as np
import open_clip

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:wisdomik/QuiltNet-B-32')
tokenizer = open_clip.get_tokenizer('hf-hub:wisdomik/QuiltNet-B-32')
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_dir = '../../datasets/LC25000/lung_colon_image_set/'


in_class = ['benign', 'adenocarcinoma', ]
in_dir = ['lung_image_sets/lung_n/', 'lung_image_sets/lung_aca/']

ood_dir = ['lung_image_sets/lung_scc/']
in_images = []
ood_images = []

model.to(device)
per_image = 150
all_gts = []
for i in range(len(in_dir)):
    _in = in_dir[i]
    sub_dir_path = os.path.join(data_dir, _in)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    in_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    #in_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
    all_gts.extend([i for x in range(per_image)])

for _ood in ood_dir:
    sub_dir_path = os.path.join(data_dir, _ood)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:300]])
    # ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
# Prepare text prompts using different class names

ind_score = []
all_preds = []
in_probs = []
for in_image in tqdm(in_images):
    image = preprocess_val(Image.open(in_image)).unsqueeze(0).to(device)
    text = tokenizer(["%s presented in image" %x for x in in_class]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        logis = (image_features @ text_features.T).cpu().numpy()
        # 计算相似度
        probs = (image_features @ text_features.T).softmax(dim=-1).cpu().numpy()
        in_probs.append(probs)
        preds = np.argmax(probs, axis=1)
    all_preds.append(preds)
    ind_score.append(-np.max(logis, axis=1)) 
all_preds = np.concatenate(all_preds)
ood_score = []
for in_image in tqdm(ood_images):
    image = preprocess_val(Image.open(in_image)).unsqueeze(0).to(device)
    text = tokenizer(["%s presented in image" %x for x in in_class]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        logis = (image_features @ text_features.T).cpu().numpy()
        # 计算相似度
        probs = (image_features @ text_features.T).softmax(dim=-1).cpu().numpy()
    ood_score.append(-np.max(logis, axis=1)) 
from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)

# Print results
print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
concat_score = [x for x in ind_score] + [x for x in ood_score]
concat_gt = [0 for x in range(len(ind_score))] + [1 for x in range(len(ood_score))]

fpr, tpr, _ = roc_curve(concat_gt, concat_score)
roc_auc = auc(fpr, tpr)

# 计算AUPR
precision, recall, _ = precision_recall_curve(concat_gt, concat_score)
aupr = average_precision_score(concat_gt, concat_score)

# 计算FPR95
# 找到tpr最接近0.95的索引
idx = np.argmin(np.abs(tpr - 0.95))
fpr95 = fpr[idx]

print(f"AUC: {roc_auc}")
print(f"AUPR: {aupr}")
print(f"FPR95: {fpr95}")

preds_probs = [x[0][1] for x in in_probs]
from sklearn.metrics import roc_auc_score
roc_auc_score(all_gts, preds_probs)

In [ ]:
import random
import os
from tqdm import tqdm
import torch
from PIL import Image
import numpy as np
import open_clip

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:wisdomik/QuiltNet-B-32')
tokenizer = open_clip.get_tokenizer('hf-hub:wisdomik/QuiltNet-B-32')
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_dir = '../../datasets/LC25000/lung_colon_image_set/'


in_class = ['benign', 'adenocarcinoma', ]
in_dir = ['lung_image_sets/lung_n/', 'lung_image_sets/lung_aca/']

ood_dir = ['imagenet/']
in_images = []
ood_images = []

model.to(device)
per_image = 150
all_gts = []
for i in range(len(in_dir)):
    _in = in_dir[i]
    sub_dir_path = os.path.join(data_dir, _in)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    in_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    #in_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
    all_gts.extend([i for x in range(per_image)])

for _ood in ood_dir:
    sub_dir_path = os.path.join(data_dir, _ood)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:300]])
    # ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
# Prepare text prompts using different class names

ind_score = []
all_preds = []
in_probs = []
for in_image in tqdm(in_images):
    image = preprocess_val(Image.open(in_image)).unsqueeze(0).to(device)
    text = tokenizer(["%s presented in image" %x for x in in_class]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        logits = (image_features @ text_features.T).cpu().numpy()
        # 计算相似度
        probs = (image_features @ text_features.T).softmax(dim=-1).cpu().numpy()
        in_probs.append(probs)
        preds = np.argmax(probs, axis=1)
    all_preds.append(preds)
    ind_score.append(-np.max(logits, axis=1)) 
all_preds = np.concatenate(all_preds)
ood_score = []
for in_image in tqdm(ood_images):
    image = preprocess_val(Image.open(in_image)).unsqueeze(0).to(device)
    text = tokenizer(["%s presented in image" %x for x in in_class]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        logits = (image_features @ text_features.T).cpu().numpy()
        # 计算相似度
        probs = (image_features @ text_features.T).softmax(dim=-1).cpu().numpy()
    ood_score.append(-np.max(logits, axis=1)) 
from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)

# Print results
print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
concat_score = [x for x in ind_score] + [x for x in ood_score]
concat_gt = [0 for x in range(len(ind_score))] + [1 for x in range(len(ood_score))]

fpr, tpr, _ = roc_curve(concat_gt, concat_score)
roc_auc = auc(fpr, tpr)

# 计算AUPR
precision, recall, _ = precision_recall_curve(concat_gt, concat_score)
aupr = average_precision_score(concat_gt, concat_score)

# 计算FPR95
# 找到tpr最接近0.95的索引
idx = np.argmin(np.abs(tpr - 0.95))
fpr95 = fpr[idx]

print(f"AUC: {roc_auc}")
print(f"AUPR: {aupr}")
print(f"FPR95: {fpr95}")

preds_probs = [x[0][1] for x in in_probs]
from sklearn.metrics import roc_auc_score
roc_auc_score(all_gts, preds_probs)

In [ ]:
import random
import os
from tqdm import tqdm
import torch
from PIL import Image
import numpy as np
import open_clip

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:wisdomik/QuiltNet-B-32')
tokenizer = open_clip.get_tokenizer('hf-hub:wisdomik/QuiltNet-B-32')
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_dir = '../../datasets/LC25000/lung_colon_image_set/'

in_class = ['benign', 'adenocarcinoma']
in_dir = ['lung_image_sets/lung_n/', 'lung_image_sets/lung_aca/']

ood_dir = ['colon_image_sets/colon_aca/', 'colon_image_sets/colon_n/']
in_images = []
ood_images = []

model.to(device)
per_image = 150
all_gts = []
for i in range(len(in_dir)):
    _in = in_dir[i]
    sub_dir_path = os.path.join(data_dir, _in)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    in_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    #in_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
    all_gts.extend([i for x in range(per_image)])

for _ood in ood_dir:
    sub_dir_path = os.path.join(data_dir, _ood)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    # ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
# Prepare text prompts using different class names

ind_score = []
all_preds = []
in_probs = []
for in_image in tqdm(in_images):
    image = preprocess_val(Image.open(in_image)).unsqueeze(0).to(device)
    text = tokenizer(["%s presented in image" %x for x in in_class]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        logits = (image_features @ text_features.T).cpu().numpy()
        # 计算相似度
        probs = (image_features @ text_features.T).softmax(dim=-1).cpu().numpy()
        in_probs.append(probs)
        preds = np.argmax(probs, axis=1)
    all_preds.append(preds)
    ind_score.append(-np.max(logits, axis=1)) 
all_preds = np.concatenate(all_preds)
ood_score = []
for in_image in tqdm(ood_images):
    image = preprocess_val(Image.open(in_image)).unsqueeze(0).to(device)
    text = tokenizer(["%s presented in image" %x for x in in_class]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        logits = (image_features @ text_features.T).cpu().numpy()
        # 计算相似度
        probs = (image_features @ text_features.T).softmax(dim=-1).cpu().numpy()
    ood_score.append(-np.max(logits, axis=1)) 
from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)

# Print results
print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
concat_score = [x for x in ind_score] + [x for x in ood_score]
concat_gt = [0 for x in range(len(ind_score))] + [1 for x in range(len(ood_score))]

fpr, tpr, _ = roc_curve(concat_gt, concat_score)
roc_auc = auc(fpr, tpr)

# 计算AUPR
precision, recall, _ = precision_recall_curve(concat_gt, concat_score)
aupr = average_precision_score(concat_gt, concat_score)

# 计算FPR95
# 找到tpr最接近0.95的索引
idx = np.argmin(np.abs(tpr - 0.95))
fpr95 = fpr[idx]

print(f"AUC: {roc_auc}")
print(f"AUPR: {aupr}")
print(f"FPR95: {fpr95}")

preds_probs = [x[0][1] for x in in_probs]
from sklearn.metrics import roc_auc_score
roc_auc_score(all_gts, preds_probs)